# Model Commands

Grundlegende Spielbefehle ausprobieren...

In [ ]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

load_dotenv(dotenv_path='../.env')

driver = GraphDatabase.driver(
    uri=os.getenv('NEO4J_URI'),
    auth=(
        os.getenv('NEO4J_USER'), 
        os.getenv('NEO4J_PASSWORD')
    )
)

print('Docker starten ;-)')
print(f'URI: {os.getenv("NEO4J_URI")}')
print(f'UI:  http://localhost:7474')

# Queryhelper
def run_queries(queries, params=None):
    
    # Ergebnisliste
    results = []

    # Liste durchgehen. Muss immer eine Liste sein, keine Normalisierung aktuell.
    for query in queries:

        # Session öffnen
        with driver.session() as session:

            # Query
            result = session.run(query, params or {})

            # Ergebnisse wegspeichern
            records = [r.data() for r in result]
            results.extend(records)

    return results

In [ ]:
# Show
show_query = [
    """
    MATCH (p:Player {id: 'player_no1'})-[:IST_IN]->(place:Location)
    RETURN place.id, place.name, place.description
    """,
    """
    MATCH (l:Location {id: 'marktplatz'})-[:ERREICHT]->(target:Location)
    RETURN target.id, target.name, target.description
    """
]

show_info = run_queries(show_query)

print(show_info[0]['place.description'])
print(show_info[1]['target.name'])

In [ ]:
# Move
move_to_taverne = [
    """
    MATCH (p:Player {id: 'player_no1'})-[old:IST_IN]->(current:Location)
    MATCH (current)-[:ERREICHT]->(target:Location {id: 'taverne'})
    DELETE old
    CREATE (p)-[:IST_IN]->(target)
    RETURN target.id, target.name, target.description
    """
]

move_to_taverne_info = run_queries(move_to_taverne)

# print(move_to_taverne_info)
print(f"Du bist jetzt in {move_to_taverne_info[0]['target.name']}.")
print(f"{move_to_taverne_info[0]['target.description']}")
print()

move_to_finsterwald = [
    """
    MATCH (p:Player {id: 'player_no1'})-[old:IST_IN]->(current:Location)
    MATCH path = shortestPath((current)-[:ERREICHT*]-(target:Location {id: 'finsterwald'}))
    WHERE path IS NOT NULL
    DELETE old
    CREATE (p)-[:IST_IN]->(target)
    RETURN target.id, target.name, target.description,
        [node IN nodes(path) | node.name] AS route_taken
    """
]

move_to_finsterwald_info = run_queries(move_to_finsterwald)

# print(move_to_finsterwald_info)
print(move_to_finsterwald_info[0]['route_taken'])
print(f"Du bist jetzt in {move_to_finsterwald_info[0]['target.name']}.")
print(f"{move_to_finsterwald_info[0]['target.description']}")
print()

move_to_marktplatz = [
    """
    MATCH (p:Player {id: 'player_no1'})-[old:IST_IN]->(current:Location)
    MATCH (current)-[:ERREICHT]->(target:Location {id: 'marktplatz'})
    DELETE old
    CREATE (p)-[:IST_IN]->(target)
    RETURN target.id, target.name, target.description
    """
]

move_to_taverne_info = run_queries(move_to_marktplatz)

# print(move_to_taverne_info)


In [ ]:
# Take
take_query = [
    """
    MATCH (p:Player {id: 'player_no1'})-[:IST_IN]->(loc:Location)
    MATCH (i:Item {id: 'schluessel'})-[old:IST_IN]->(loc)
    DELETE old
    CREATE (p)-[:TRÄGT]->(item)
    RETURN i.name, loc.name
    """
]
take_info = run_queries(take_query)
print (take_info)

if not take_info:
    print('nope!')

In [ ]:
# Inventar
inventory_show = [
    """
    MATCH (p:Player {id: 'player_no1'})-[:TRÄGT]->(i:Item)
    RETURN i.name
    """
]

inventory_info = run_queries(inventory_show)
print(inventory_info)